In [2]:
import os
import sys
import pandas as pd
from collections import defaultdict, Counter

In [6]:
# ============== PARAMETERS ==============
MAX_INTERVAL_GAP = 300000  # Maximum gap (bp) for merging intervals on same chromosome
# =========================================

# Read the three tables
tcga_df = pd.read_csv('TCGA_result_table.tsv', sep='\t')
pcawg_df = pd.read_csv('PCAWG_result_table.tsv', sep='\t')
hmf_df = pd.read_csv('HMF_result_table.tsv', sep='\t')

# Add source column to track origin
tcga_df['Source'] = 'TCGA'
pcawg_df['Source'] = 'PCAWG'
hmf_df['Source'] = 'HMF'

# Combine all three dataframes
combined_df = pd.concat([tcga_df, pcawg_df, hmf_df], ignore_index=True)

print(f"Total rows: {len(combined_df)}")
print(f"TCGA: {len(tcga_df)}, PCAWG: {len(pcawg_df)}, HMF: {len(hmf_df)}")

# ============== SAMPLE-LEVEL ANALYSIS ==============
print(f"\n{'='*60}")
print(f"=== Sample-Level Analysis ===")
print(f"{'='*60}")

# Count unique samples in each dataset
tcga_samples = tcga_df['Sample name'].nunique()
pcawg_samples = pcawg_df['Sample name'].nunique()
hmf_samples = hmf_df['Sample name'].nunique()
total_samples = combined_df['Sample name'].nunique()

print(f"\nUnique samples per dataset:")
print(f"  TCGA:  {tcga_samples:5d} samples")
print(f"  PCAWG: {pcawg_samples:5d} samples")
print(f"  HMF:   {hmf_samples:5d} samples")
print(f"  Total: {total_samples:5d} samples")

# Function to count ecDNA per sample for each dataset
def analyze_ecdna_per_sample(df, dataset_name):
    """Count ecDNA per sample and report statistics"""
    # Filter to ecDNA only
    ecdna_only = df[df['Classification'] == 'ecDNA']
    
    # Count ecDNA per sample
    ecdna_counts = ecdna_only.groupby('Sample name').size()
    
    # Get all unique samples (including those with 0 ecDNA)
    all_samples = df['Sample name'].unique()
    
    # Create a complete count series with 0 for samples without ecDNA
    complete_counts = pd.Series(0, index=all_samples)
    complete_counts.update(ecdna_counts)
    
    # Count samples by number of ecDNA
    samples_0_ecdna = (complete_counts == 0).sum()
    samples_1_ecdna = (complete_counts == 1).sum()
    samples_multiple_ecdna = (complete_counts > 1).sum()
    
    total = len(all_samples)
    
    print(f"\n--- {dataset_name} ---")
    print(f"Samples with 0 ecDNA:        {samples_0_ecdna:5d} ({100*samples_0_ecdna/total:5.2f}%)")
    print(f"Samples with 1 ecDNA:        {samples_1_ecdna:5d} ({100*samples_1_ecdna/total:5.2f}%)")
    print(f"Samples with multiple ecDNA: {samples_multiple_ecdna:5d} ({100*samples_multiple_ecdna/total:5.2f}%)")
    print(f"Total samples: {total}")
    
    # Distribution of ecDNA counts
    ecdna_distribution = complete_counts.value_counts().sort_index()
    print(f"\nDetailed distribution:")
    print(f"ecDNA count | # Samples | Percentage")
    print("-" * 42)
    for count in sorted(ecdna_distribution.index):
        if count <= 10:
            n_samples = ecdna_distribution[count]
            pct = 100 * n_samples / total
            print(f"{count:11d} | {n_samples:9d} | {pct:6.2f}%")
    
    # 11+ group
    samples_11_plus = (complete_counts > 10).sum()
    if samples_11_plus > 0:
        pct = 100 * samples_11_plus / total
        print(f"{'11+':>11s} | {samples_11_plus:9d} | {pct:6.2f}%")
    
    return complete_counts

# Analyze each dataset
print(f"\n{'='*60}")
print(f"=== ecDNA per Sample by Dataset ===")
print(f"{'='*60}")

tcga_ecdna_counts = analyze_ecdna_per_sample(tcga_df, "TCGA")
pcawg_ecdna_counts = analyze_ecdna_per_sample(pcawg_df, "PCAWG")
hmf_ecdna_counts = analyze_ecdna_per_sample(hmf_df, "HMF")

# Combined analysis
print(f"\n{'='*60}")
print(f"=== Combined Analysis (All Datasets) ===")
print(f"{'='*60}")
combined_ecdna_counts = analyze_ecdna_per_sample(combined_df, "Combined")

# Function to parse location string into list of tuples (chr, start, end)
def parse_location(loc_str):
    """Parse location string like "['1:100-200', '2:300-400']" into list of (chr, start, end)"""
    if pd.isna(loc_str) or loc_str == '':
        return []
    
    # Remove brackets and quotes
    loc_str = loc_str.strip("[]'\"")
    
    # Split by comma and clean up
    intervals = [x.strip().strip("'\"") for x in loc_str.split("',") if x.strip()]
    
    parsed = []
    for interval in intervals:
        if ':' in interval and '-' in interval:
            chrom, pos = interval.split(':')
            start, end = pos.split('-')
            parsed.append((chrom.strip(), int(start), int(end)))
    
    return parsed

# Function to merge intervals within distance threshold on same chromosome
def merge_intervals_by_chr(intervals, max_gap=MAX_INTERVAL_GAP):
    """
    Merge intervals on the same chromosome that are within max_gap bp of each other.
    Returns dict: {chr: [(start1, end1), (start2, end2), ...]}
    """
    # Group by chromosome
    by_chr = defaultdict(list)
    for chrom, start, end in intervals:
        by_chr[chrom].append((start, end))
    
    # Merge intervals on each chromosome
    merged_by_chr = {}
    for chrom, chr_intervals in by_chr.items():
        # Sort by start position
        chr_intervals.sort()
        
        merged = []
        for start, end in chr_intervals:
            if merged and start - merged[-1][1] <= max_gap:
                # Merge with previous interval
                merged[-1] = (merged[-1][0], max(merged[-1][1], end))
            else:
                # Start new interval
                merged.append((start, end))
        
        merged_by_chr[chrom] = merged
    
    return merged_by_chr

# Apply parsing and merging
def process_location(loc_str):
    """Process location string and return merged intervals by chromosome"""
    intervals = parse_location(loc_str)
    if not intervals:
        return {}
    return merge_intervals_by_chr(intervals)

# Add merged intervals column
combined_df['Merged_intervals'] = combined_df['Location'].apply(process_location)

# Function to format merged intervals back to string
def format_merged_intervals(merged_dict):
    """Convert merged intervals dict to readable string"""
    if not merged_dict:
        return ''
    
    result = []
    for chrom in sorted(merged_dict.keys()):
        intervals = merged_dict[chrom]
        for start, end in intervals:
            result.append(f"{chrom}:{start}-{end}")
    
    return str(result)

combined_df['Merged_intervals_str'] = combined_df['Merged_intervals'].apply(format_merged_intervals)

# Calculate interval counts per chromosome for each ecDNA
def count_intervals_per_chr(merged_dict):
    """Return list of interval counts per chromosome"""
    if not merged_dict:
        return []
    return [len(intervals) for intervals in merged_dict.values()]

combined_df['Intervals_per_chr'] = combined_df['Merged_intervals'].apply(count_intervals_per_chr)

# Calculate number of chromosomes involved
def count_chromosomes(merged_dict):
    """Return number of distinct chromosomes"""
    return len(merged_dict)

combined_df['Num_chromosomes'] = combined_df['Merged_intervals'].apply(count_chromosomes)

# ============== FILTER FOR ecDNA ONLY ==============
# Filter to only ecDNA features for the analysis
ecdna_df = combined_df[combined_df['Classification'] == 'ecDNA'].copy()

print(f"\n{'='*60}")
print(f"=== Feature-Level Statistics ===")
print(f"{'='*60}")
print(f"\nTotal features: {len(combined_df)}")
print(f"ecDNA features: {len(ecdna_df)}")
print(f"Other classifications: {len(combined_df) - len(ecdna_df)}")

# ============== MULTI-CHROMOSOMAL ANALYSIS ==============
print(f"\n{'='*60}")
print(f"=== Multi-chromosomal ecDNA Analysis ===")
print(f"{'='*60}")

multi_chr_count = (ecdna_df['Num_chromosomes'] > 1).sum()
single_chr_count = (ecdna_df['Num_chromosomes'] == 1).sum()

print(f"\nSingle-chromosome ecDNAs: {single_chr_count} ({100*single_chr_count/len(ecdna_df):.2f}%)")
print(f"Multi-chromosomal ecDNAs:  {multi_chr_count} ({100*multi_chr_count/len(ecdna_df):.2f}%)")
print(f"Total ecDNAs: {len(ecdna_df)}")

# Frequency distribution of number of chromosomes
chr_count_freq = Counter(ecdna_df['Num_chromosomes'])

# Cap at 10
chr_count_freq_capped = {}
count_11_plus = 0
for count, freq in chr_count_freq.items():
    if count <= 10:
        chr_count_freq_capped[count] = freq
    else:
        count_11_plus += freq

if count_11_plus > 0:
    chr_count_freq_capped['11+'] = count_11_plus

print("\n=== Number of chromosomes per ecDNA ===")
print("Chromosome count | Frequency | Percentage")
print("-" * 45)

# Print 1-10 first
for count in range(1, 11):
    if count in chr_count_freq_capped:
        freq = chr_count_freq_capped[count]
        pct = 100 * freq / len(ecdna_df)
        print(f"{count:16d} | {freq:9d} | {pct:6.2f}%")

# Print 11+ if it exists
if '11+' in chr_count_freq_capped:
    freq = chr_count_freq_capped['11+']
    pct = 100 * freq / len(ecdna_df)
    print(f"{'11+':>16s} | {freq:9d} | {pct:6.2f}%")

# ============== MULTI-INTERVAL ANALYSIS ==============
print(f"\n{'='*60}")
print(f"=== Multi-interval (same chromosome) ecDNA Analysis ===")
print(f"{'='*60}")

# Count how many ecDNAs have multiple intervals from same chromosome
multi_interval_count = sum(any(count > 1 for count in counts) 
                           for counts in ecdna_df['Intervals_per_chr'] if counts)

print(f"\necDNAs with multiple intervals from same chromosome: {multi_interval_count}")
print(f"ecDNAs with single interval per chromosome: {len(ecdna_df) - multi_interval_count}")
print(f"Total ecDNAs: {len(ecdna_df)}")
print(f"Multi-interval percentage: {100*multi_interval_count/len(ecdna_df):.2f}%")

# Create frequency distribution of interval counts
# Each ecDNA contributes to multiple bins if it has different chromosomes
all_interval_counts = []
for counts in ecdna_df['Intervals_per_chr']:
    all_interval_counts.extend(counts)

interval_freq = Counter(all_interval_counts)

# Cap at 10, sum everything 11+ together
interval_freq_capped = {}
count_11_plus = 0
for count, freq in interval_freq.items():
    if count <= 10:
        interval_freq_capped[count] = freq
    else:
        count_11_plus += freq

if count_11_plus > 0:
    interval_freq_capped['11+'] = count_11_plus

print("\n=== Frequency of interval counts per chromosome (ecDNA only) ===")
print("(Each chromosome from each ecDNA counted separately)")
print(f"Max interval gap for merging: {MAX_INTERVAL_GAP:,} bp")
print("\nInterval count | Frequency | Percentage")
print("-" * 45)
total_chr_instances = sum(interval_freq.values())

# Print 1-10 first
for count in range(1, 11):
    if count in interval_freq_capped:
        freq = interval_freq_capped[count]
        pct = 100 * freq / total_chr_instances
        print(f"{count:14d} | {freq:9d} | {pct:6.2f}%")

# Print 11+ if it exists
if '11+' in interval_freq_capped:
    freq = interval_freq_capped['11+']
    pct = 100 * freq / total_chr_instances
    print(f"{'11+':>14s} | {freq:9d} | {pct:6.2f}%")

print(f"\nTotal chromosome instances: {total_chr_instances}")

# More detailed breakdown per ecDNA
print("\n=== Distribution of ecDNAs by their interval patterns ===")
pattern_counter = Counter()
for counts in ecdna_df['Intervals_per_chr']:
    if counts:
        # Sort the counts to create a pattern signature
        pattern = tuple(sorted(counts))
        pattern_counter[pattern] += 1

print("\nPattern (sorted interval counts) | Frequency")
print("-" * 50)
for pattern, freq in pattern_counter.most_common(20):
    print(f"{str(pattern):30s} | {freq}")

# Save the combined dataframe with merged intervals (includes all features)
combined_df.to_csv('combined_with_merged_intervals.tsv', sep='\t', index=False)
print(f"\n{'='*60}")
print(f"=== Output Files ===")
print(f"{'='*60}")
print("Saved combined data (all features) to: combined_with_merged_intervals.tsv")

# Optionally save ecDNA-only subset
ecdna_df.to_csv('ecdna_with_merged_intervals.tsv', sep='\t', index=False)
print("Saved ecDNA-only data to: ecdna_with_merged_intervals.tsv")

Total rows: 13323
TCGA: 230, PCAWG: 3939, HMF: 9154

=== Sample-Level Analysis ===

Unique samples per dataset:
  TCGA:    101 samples
  PCAWG:  2095 samples
  HMF:    4170 samples
  Total:  6366 samples

=== ecDNA per Sample by Dataset ===

--- TCGA ---
Samples with 0 ecDNA:           77 (76.24%)
Samples with 1 ecDNA:           16 (15.84%)
Samples with multiple ecDNA:     8 ( 7.92%)
Total samples: 101

Detailed distribution:
ecDNA count | # Samples | Percentage
------------------------------------------
          0 |        77 |  76.24%
          1 |        16 |  15.84%
          2 |         5 |   4.95%
          3 |         1 |   0.99%
          4 |         1 |   0.99%
          6 |         1 |   0.99%

--- PCAWG ---
Samples with 0 ecDNA:         1695 (80.91%)
Samples with 1 ecDNA:          259 (12.36%)
Samples with multiple ecDNA:   141 ( 6.73%)
Total samples: 2095

Detailed distribution:
ecDNA count | # Samples | Percentage
------------------------------------------
          0 |  